# 0. Initialized project

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv
/kaggle/input/homecredit-feature-group/df_ln_grp.parquet
/kaggle/input/homecredit-feature-group/pcb_info_conso.parquet
/kaggle/input/homecredit-feature-group/df_ts_amt_n_count_fn.parquet
/kaggle/input/homecredit-feature-group/ins_pmt_feature.parquet
/kaggle/input/homecredit-feature-group/main_df.parquet
/kaggle/input/homecredit-feature-group/HomeCredit_feature_group.xlsx
/kaggle/in

In [2]:
import sklearn
# import statsmodels
# import optbinning
# import mlxtend

from tqdm import tqdm

import lightgbm
# import xgboost

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_colwidth', None)

In [3]:
"""
/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv
"""

'\n/kaggle/input/home-credit-default-risk/sample_submission.csv\n/kaggle/input/home-credit-default-risk/bureau_balance.csv\n/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv\n/kaggle/input/home-credit-default-risk/application_train.csv\n/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv\n/kaggle/input/home-credit-default-risk/application_test.csv\n/kaggle/input/home-credit-default-risk/previous_application.csv\n/kaggle/input/home-credit-default-risk/credit_card_balance.csv\n/kaggle/input/home-credit-default-risk/installments_payments.csv\n/kaggle/input/home-credit-default-risk/bureau.csv\n'

In [4]:
os.chdir('/kaggle/input/homecredit-feature-group')
feature_group = pd.read_excel('HomeCredit_feature_group.xlsx')

In [5]:
os.chdir('/kaggle/input/home-credit-default-risk')
#sample
sample_submission = pd.read_csv('sample_submission.csv', encoding='utf-8')
#bureau info
bureau = pd.read_csv('bureau.csv', encoding='utf-8')
bureau_balance = pd.read_csv('bureau_balance.csv', encoding='utf-8')
#main info
application_train = pd.read_csv('application_train.csv', encoding='utf-8')
application_test = pd.read_csv('application_test.csv', encoding='utf-8')
#past info
previous_application = pd.read_csv('previous_application.csv', encoding='utf-8')
#balance
pos_cash_balance = pd.read_csv('POS_CASH_balance.csv', encoding='utf-8')
#card info
credit_card_balance = pd.read_csv('credit_card_balance.csv', encoding='utf-8')
#payment info
installments_payments = pd.read_csv('installments_payments.csv', encoding='utf-8')

In [6]:
id_col = list(feature_group[feature_group['GROUP'].isin(['ID', 'LABEL'])]['Row'].unique())
feature_col = list(feature_group[~feature_group['GROUP'].isin(['ID', 'LABEL'])]['Row'].unique())

# 1. Exploratory data analysis

In [7]:
feature_group['GROUP'].value_counts()

GROUP
Credit info       65
Housing           55
Demo              31
Credit balance    18
POS               15
ID                13
Overdue            7
Enquiries          6
Credit score       3
Credit Info        2
Payment            2
LABEL              1
Income             1
Name: count, dtype: int64

In [ ]:
id_col = list(feature_group[feature_group['GROUP'].isin(['ID', 'LABEL'])]['Row'].unique())
feature_col = list(feature_group[~feature_group['GROUP'].isin(['ID', 'LABEL'])]['Row'].unique())

In [ ]:
application_train.head(10)

In [ ]:
application_train.NAME_CONTRACT_TYPE.value_counts().index

In [ ]:
application_train[['NAME_FAMILY_STATUS','TARGET']].groupby(['NAME_FAMILY_STATUS'], as_index=False).mean()

In [ ]:
application_train[['NAME_FAMILY_STATUS']].value_counts()

In [ ]:
application_train[['NAME_FAMILY_STATUS','TARGET']].groupby(['NAME_FAMILY_STATUS']).mean()

In [ ]:
application_train[['NAME_FAMILY_STATUS','FONDKAPREMONT_MODE','TARGET']].value_counts()

In [ ]:
application_train[['NAME_FAMILY_STATUS', 'FONDKAPREMONT_MODE','TARGET']]

In [ ]:
application_train.pivot_table(values='TARGET',
                  columns=['NAME_FAMILY_STATUS'],
                  aggfunc=['count'],
                  dropna=False,
         index=['FONDKAPREMONT_MODE'])

In [ ]:
application_train.pivot_table(values='TARGET',
                  aggfunc=['sum', 'count', 'mean'],
                  dropna=False,
         index=['NAME_FAMILY_STATUS'])

In [7]:
def plot_stats(feature:str, label:str, df, horizontal_layout, label_rotation):
    #prepare data
    temp_count = df[feature].value_counts()
    df_count = pd.DataFrame({feature:temp_count.index, 'No contracts':temp_count.values})
    df_perc = df[[feature, label]].groupby([feature], as_index=False).mean()
    df_perc.sort_values(by=label, ascending=False, inplace=True)
    
    #initialize subplot
    #horizontal layout
    if horizontal_layout:
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
    else:
        fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(12,14))
    sns.set_color_codes('pastel')
    s1 = sns.barplot(ax=ax1, x=feature, y='No contracts', order=df_perc[feature], data=df_count)
    s2 = sns.barplot(ax=ax2, x=feature, y=label, order=df_perc[feature], data=df_perc)
    
    #tickmark flip
    if label_rotation:
        s1.set_xticklabels(s1.get_xticklabels(), rotation=90)
        s2.set_xticklabels(s2.get_xticklabels(), rotation=90)
    
    #set up chart
    plt.ylabel('Percent of target with 1', fontsize=10)
#     plt.tick_params(axis='both', which='major', labelsize=10)
    plt.show()

In [8]:
def plot_distribution(df, feature, bins=100):
    plt.figure(figsize=(10,6))
    plt.title("Distribution of %s"%feature)
    sns.histplot(df[feature].dropna(), color='green', kde=True, bins=bins)
    plt.show()

In [9]:
def plot_distribution_target(df, feature_list, label, nrows=3):
    #data
    t1 = df.loc[df[label] != 0]
    t0 = df.loc[df[label] == 0]
    #get frame
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(nrows=nrows, ncols=2, figsize=(12, 6*nrows))
    
    i=0
    for feature in feature_list:
        i += 1
        plt.subplot(nrows,2,i)
        sns.kdeplot(t1[feature], bw_method=0.01, label='TARGET [1]')
        sns.kdeplot(t0[feature], bw_method=0.01, label='TARGET [0]')
        plt.ylabel('Density plot', fontsize=12)
        plt.xlabel(feature, fontsize=12)
        locs, labels = plt.xticks()
        plt.tick_params(axis='both', which='major', labelsize=10)
    plt.legend()
    plt.show()

In [ ]:
plot_stats('FONDKAPREMONT_MODE','TARGET', application_train, True, True)

In [ ]:
plot_stats('OCCUPATION_TYPE','TARGET', application_train, True, True)

In [ ]:
for i in range(2,22):
    plot_stats(f'FLAG_DOCUMENT_{i}','TARGET', application_train, True, True)

In [ ]:
application_train.info(verbose=True)

In [ ]:
plot_distribution(application_train, 'AMT_CREDIT', bins=30)

In [ ]:
feature_list = ['AMT_REQ_CREDIT_BUREAU_YEAR','CNT_CHILDREN',
                'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
                'AMT_INCOME_TOTAL','AMT_REQ_CREDIT_BUREAU_YEAR',
                'LIVINGAPARTMENTS_AVG'
               ]
plot_distribution_target(application_train, feature_list, 'TARGET', nrows=4)

In [ ]:
id_list = feature_group[feature_group['GROUP'].isin(['ID','LABEL'])]['Row']
feature_list = list(set(feature_group['Row']) - set(id_list))

In [ ]:
len(feature_list)

In [ ]:
plot_distribution(bureau, 'AMT_ANNUITY', bins=30)

In [ ]:
plot_distribution(bureau, 'AMT_CREDIT_SUM_LIMIT', bins=30)

In [ ]:
bureau_balance.head()

In [ ]:
bureau_balance['STATUS'].value_counts()

In [ ]:
pos_cash_balance.head()

In [ ]:
application_train.info(verbose=True, show_counts=True)

In [ ]:
application_train['FLAG_DOCUMENT_10'].isna().sum()

In [ ]:
filter1_comp['app']

# 2 Feature engineering

In [ ]:
# os.chdir('/kaggle/working')

## main feature - income, credit

In [10]:
def distinct_count(df):
    most_count_lst = {'col':[],'most_count':[], 'null_count':[]}
    total_count = len(df)
    for col in df.columns:
        most_count = df[col].value_counts(sort=True, dropna=False).iloc[0]
        null_count = df[col].isna().sum()
        
        cover_pct = most_count/total_count
        null_pct = null_count/total_count
        
        most_count_lst['col'].append(col)
        most_count_lst['most_count'].append(cover_pct)
        most_count_lst['null_count'].append(null_pct)
    
    return pd.DataFrame(most_count_lst)

df_list = [application_train, previous_application, pos_cash_balance, credit_card_balance, installments_payments, bureau, bureau_balance]
feat_store = ['app', 'prev', 'pos', 'cc', 'inst', 'bu', 'bu_bal']

filter1_comp = {}

for df, st in zip(df_list, feat_store):
    filter1 = distinct_count(df)
    drop_most_count = list(set(filter1[filter1['most_count']>=0.8]['col'])-set(id_col))
    drop_most_null = list(set(filter1[filter1['null_count']>=0.6]['col'])-set(id_col))
    filter1_comp[st] = drop_most_count + drop_most_null
    print(st)

app
prev
pos
cc
inst
bu
bu_bal


In [11]:
# id_col = list(feature_group[feature_group['GROUP'].isin(['ID', 'LABEL'])]['Row'].unique())
# feature_col = list(feature_group[~feature_group['GROUP'].isin(['ID', 'LABEL'])]['Row'].unique())

In [12]:
filter1_comp['bu']

['AMT_CREDIT_SUM_OVERDUE',
 'CNT_CREDIT_PROLONG',
 'CREDIT_DAY_OVERDUE',
 'CREDIT_CURRENCY',
 'AMT_CREDIT_MAX_OVERDUE',
 'AMT_ANNUITY']

In [13]:
id_col

['SK_ID_CURR', 'TARGET', 'SK_BUREAU_ID', 'SK_ID_PREV ', 'NAME_CONTRACT_TYPE']

In [14]:
### PROCESS ###
def application_feature(df):
    df['AGE'] = round(df['DAYS_BIRTH']*-1/365,0)
    df['YEAR_WORKING'] = round(df['DAYS_EMPLOYED']*-1/365,0)
    df['inc_ovr_credit'] = df['AMT_INCOME_TOTAL']/df['AMT_CREDIT']
    df['inc_ovr_annuity'] = df['AMT_INCOME_TOTAL']/df['AMT_ANNUITY']
    df['inc_ovr_good'] = df['AMT_INCOME_TOTAL']/df['AMT_GOODS_PRICE']
    return df

In [ ]:
application_train = (
    application_train.pipe(application_feature)
    )

In [ ]:
application_train['inc_ovr_good'].hist(range=(0,2))

## TS dpd, loan_term

In [15]:
from functools import reduce

def time_travel(df, mth):
    """
    DAYS_CREDIT --> LxM
    """
    filter_time = (df['LxM']*-1) <= mth
    return df[filter_time]
   
# customer function
def q20(x):
    return x.quantile(0.2)

def q80(x):
    return x.quantile(0.8)

def average(x, mth_counts):
    """
    x = pd.Series
    """
    return x.sum()/mth_counts

    
def agg_cal(input_df:pd.DataFrame, 
            group_col:list, 
            val:str,
            agg_fn:list,
           ):    
    
    group_df = input_df[[val]+group_col].groupby(group_col, dropna=False)
    
    df = group_df.agg({val:agg_fn})
    df.columns = agg_fn
    all_func = agg_fn
    function_name = [fn.__name__ if callable(fn) else fn for fn in agg_fn]
    rename_cols = [val+'_'+F for F in (function_name)]
    df.columns = rename_cols
    df.reset_index(inplace=True)
    
    return df


def generate_feature_lxm(df, 
                         group_col:list, 
                         val_col:list, 
                         agg_fn:list,  
                         LxM:list):
    """
        required columns: ['LxM']
    """
    result_lst = []QD
    for val in tqdm(val_col):
        for mth_counts in LxM: 
            df_lxm = time_travel(df, mth_counts)
            result = agg_cal(df_lxm, group_col, val, agg_fn)
            feat_lst = list(set(result.columns) - set(group_col))
            rename_lst = {f:f+f'_l{mth_counts}m' for f in feat_lst}
            result.rename(columns=rename_lst, inplace=True)
            result_lst.append(result)
        
    final_df = reduce(lambda  left,right: pd.merge(left, right, on=group_col, how='outer'), result_lst)
    return final_df

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
df_pcb = bureau[bureau['SK_ID_CURR'].isin(application_train['SK_ID_CURR'].unique())]
df_pcb['LxM'] = df_pcb['DAYS_CREDIT']/30.5
df_pcb['loan_term'] = df_pcb['AMT_CREDIT_SUM']/df_pcb['AMT_ANNUITY']

In [18]:
len(application_train)

307511

## TS bureau - amt, limit, term

In [ ]:
len(df_pcb)

In [ ]:
df_pcb['CREDIT_ACTIVE'].value_counts()

In [19]:
def pcb_ts_info(df_pcb):
    active = df_pcb['CREDIT_ACTIVE']=='Active'
    close = df_pcb['CREDIT_ACTIVE']=='Closed'
    df_active = df_pcb[active]
    df_close = df_pcb[close]

    group_col = ['SK_ID_CURR']
    val_col = ['AMT_CREDIT_SUM','AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_DEBT', 'loan_term']
    agg_fn = ['mean','min','max','sum','count', 'std', 'var']
    LxM = [3,6,9,12,15,24,36]

    pcb_ts_all = generate_feature_lxm(df_pcb, group_col, val_col, agg_fn, LxM)
    pcb_ts_active = generate_feature_lxm(df_active, group_col, val_col, agg_fn, LxM)
    pcb_ts_closed = generate_feature_lxm(df_close, group_col, val_col, agg_fn, LxM)

    pcb_ts_active.columns = [col+'_active' for col in pcb_ts_active.columns]
    pcb_ts_closed.columns = [col+'_close' for col in pcb_ts_closed.columns]
    
    result_lst = [pcb_ts_all, pcb_ts_active, pcb_ts_closed]
    
    final_df = reduce(lambda  left,right: pd.merge(left, right, on=group_col, how='outer'), result_lst)
    return final_df

In [17]:
pcb_ts_total = pcb_ts_info(df_pcb)

100%|██████████| 4/4 [00:01<00:00,  2.55it/s]


In [ ]:
prod_group = ['Consumer credit', 'Credit card', 'Car loan', 'Mortgage']
pcb_ts_prod = {}
for prod in prod_group:
    print(f'Processing {prod}')
    df_prod = df_pcb[df_pcb['CREDIT_TYPE'] == prod].copy()
    pcb_ts_prod[prod] = pcb_ts_info(df_prod)

In [ ]:
rename_prod = ['csm', 'card', 'car', 'mg']

for prod, name in zip(prod_group, rename_prod):
    df_to_rename = pcb_ts_prod[prod]
    df_to_rename.columns = [col+'_'+name for col in df_to_rename.columns] 
    print(prod)

In [ ]:
to_merge = [i.reset_index() for i in pcb_ts_prod.values()]
to_merge.append(pcb_ts_total)

del pcb_ts_prod
del pcb_ts_total

In [ ]:
pcb_info_conso = reduce(lambda  left,right: pd.merge(left, right, on='SK_ID_CURR', how='outer'), to_merge)

In [ ]:
filter1a = distinct_count(pcb_info_conso)
drop_most_count = list(set(filter1a[filter1a['most_count']>=0.8]['col'])-set(id_col))
drop_most_null = list(set(filter1a[filter1a['null_count']>=0.6]['col'])-set(id_col))

In [ ]:
keep = set(pcb_info_conso.columns) - set(drop_most_count + drop_most_null)

In [ ]:
pcb_info_conso_ = pcb_info_conso[list(keep)]
del pcb_info_conso

In [ ]:
application_train.to_parquet(r'main_df.parquet')
pcb_info_conso_.to_parquet(r'pcb_info_conso.parquet')

#### Checkpoint

In [20]:
#Check point
os.chdir(r'/kaggle/input/homecredit-feature-group')
application_train = pd.read_parquet(r'main_df.parquet')
pcb_info_conso_ = pd.read_parquet(r'pcb_info_conso.parquet')

In [ ]:
application_train.head()

In [ ]:
application_train.shape

## TS bureau dpd

In [ ]:
df_pcb = bureau[bureau['SK_ID_CURR'].isin(application_train['SK_ID_CURR'].unique())]
df_pcb['LxM'] = df_pcb['DAYS_CREDIT']/30.5
df_pcb['loan_term'] = df_pcb['AMT_CREDIT_SUM']/df_pcb['AMT_ANNUITY']

In [ ]:
condition = (df_pcb['CREDIT_DAY_OVERDUE'] <=10) & (df_pcb['CREDIT_DAY_OVERDUE'] >= 1)
df_pcb['is_1_10dpd'] = np.where(condition, 1, 0)

condition = (df_pcb['CREDIT_DAY_OVERDUE'] <=30) & (df_pcb['CREDIT_DAY_OVERDUE'] >= 10)
df_pcb['is_10_30dpd'] = np.where(condition, 1, 0)

condition = (df_pcb['CREDIT_DAY_OVERDUE'] > 30)
df_pcb['is_gt30dpd'] = np.where(condition, 1, 0)

In [21]:
def pcb_dpd(df_pcb):
    group_col = ['SK_ID_CURR']
    val_col = ['CREDIT_DAY_OVERDUE', 'is_1_10dpd', 'is_10_30dpd', 'is_gt30dpd']
    agg_fn = ['max']
    LxM = [3,6,9,12,15,24,36, 48,60,72,84]

    pcb_dpd_ts = generate_feature_lxm(df_pcb, group_col, val_col, agg_fn, LxM)
    return pcb_dpd_ts

In [ ]:
df_pcb_dpd = pcb_dpd(df_pcb)
df_pcb_dpd = df_pcb_dpd.fillna(0).reset_index()

In [ ]:
df_pcb['active_mths_remain'] = df_pcb[df_pcb['CREDIT_ACTIVE']=='Active']['DAYS_CREDIT_ENDDATE']/30.5
cond1 = (df_pcb['active_mths_remain']<0)
df_pcb['active_mths_remain'] = np.where(cond1, 0, df_pcb['active_mths_remain'])  
df_pcb['active_mths_remain'] = df_pcb['active_mths_remain'].fillna(0)

In [ ]:
df_pcb['card_utl_rate'] = df_pcb['AMT_CREDIT_SUM_DEBT']/df_pcb['AMT_CREDIT_SUM_LIMIT']
df_pcb['card_utl_rate'] = np.where((df_pcb['CREDIT_TYPE']=='Credit card') & (df_pcb['card_utl_rate']!=np.inf),df_pcb['card_utl_rate'],np.nan)

In [ ]:
df_pcb.head()

In [22]:
def pcb_credit_info1(df_pcb):
    group_col = ['SK_ID_CURR']
    val_col = ['active_mths_remain', 'card_utl_rate']
    agg_fn = ['mean', 'max']
    LxM = [3,6,9,12,15,24,36, 48,60,72,84]

    ts = generate_feature_lxm(df_pcb, group_col, val_col, agg_fn, LxM)
    return ts

In [ ]:
df_pcb_cr1 = pcb_credit_info1(df_pcb)
df_pcb_cr1 = df_pcb_cr1.reset_index()

In [ ]:
df_pcb_dpd.head()

In [ ]:
df_pcb_cr1.head()

In [ ]:
df_pcb_cr = pd.merge(df_pcb_dpd, df_pcb_cr1, how='outer', on='SK_ID_CURR')

In [ ]:
filter_pcb_cr = distinct_count(df_pcb_cr)
drop_most_count = list(set(filter_pcb_cr[filter_pcb_cr['most_count']>=0.9]['col'])-set(id_col))
drop_most_null = list(set(filter_pcb_cr[filter_pcb_cr['null_count']>=0.8]['col'])-set(id_col))
to_drop = drop_most_count+drop_most_null

In [ ]:
len(set(to_drop))

In [ ]:
len(df_pcb_cr.columns)

In [ ]:
df_pcb_cr_fn = df_pcb_cr[list(set(df_pcb_cr.columns) - set(to_drop))]

In [ ]:
df_pcb_cr_fn.info(verbose=True, show_counts=True)

In [ ]:
df_pcb_cr_fn.to_parquet(r'data/df_pcb_cr_fn.parquet')

## TS bureau contract_level - type, count status

In [ ]:
df_ts = pd.merge(df_pcb[['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_TYPE', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_LIMIT']], 
         bureau_balance, 
         how='inner', 
         on='SK_ID_BUREAU')

In [23]:
def replace_prod(row, prod_group):
    if row in (prod_group):
        return row
    else:
        return 'Others'

In [ ]:
prod_group = ['Consumer credit', 'Credit card', 'Car loan', 'Mortgage']
df_ts['CREDIT_TYPE'] = df_ts['CREDIT_TYPE'].apply(lambda row: replace_prod(row, prod_group))
df_ts['STATUS'].fillna('X', inplace=True)

In [ ]:
df_ts['CREDIT_TYPE'].value_counts()

In [ ]:
df_ts['STATUS'].value_counts(dropna=False)

In [ ]:
df_ts['is_active'] = np.where(df_ts['STATUS'].isin(['X', 'C']), 0, 1)
df_ts['ln_grp'] = np.where(df_ts['STATUS'].isin(['X', 'C']), 0, df_ts['STATUS'])
df_ts['ln_grp'] = df_ts['ln_grp'].astype(int)
df_ts['LxM'] = df_ts['MONTHS_BALANCE']

In [ ]:
# check bureau_id trong LxM có tồn tại process_month active
# đếm distinct bureau_id theo loại HĐ - sum theo ID

In [ ]:
group_col = ['SK_ID_CURR']
val_col = ['ln_grp']
agg_fn = ['max']
LxM = [3,6,9]
df_ln_grp = generate_feature_lxm(df_ts,
                    group_col=group_col,
                    val_col=val_col,
                    agg_fn=agg_fn,
                    LxM=LxM)

df_ln_grp = df_ln_grp.reset_index()

In [ ]:
df_ln_grp.to_parquet(r'data/df_ln_grp.parquet')

## TS bureau contract_level - count, amt per type

In [24]:
from functools import reduce

def cal_ts(ts_ft, pref, lxm_counts):
    prod_ts = ts_ft[['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_TYPE', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_LIMIT']].groupby('SK_ID_BUREAU').max().reset_index()
        
    count_type = pd.pivot_table(prod_ts, values='SK_ID_BUREAU', index=['SK_ID_CURR'], columns=['CREDIT_TYPE'], aggfunc="count").reset_index()
    count_type.columns = ['SK_ID_CURR'] + ['no_'+ i for i in count_type.columns[1:]+"_"+pref+"_" + f'l{lxm_counts}m']
    
    amt_type = pd.pivot_table(prod_ts, values=['AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_LIMIT'], index=['SK_ID_CURR'], columns=['CREDIT_TYPE'], aggfunc="sum").reset_index()
    col_to_rename = {'AMT_CREDIT_SUM':'amt', 
                             'AMT_CREDIT_SUM_LIMIT':'lmt',
                            'Car loan':'car',
                            'Consumer credit':'cons',
                            'Credit card':'cc',
                            'Mortgage':'mg',
                            'Others':'others'}
    amt_type = amt_type.rename(columns=col_to_rename)
    amt_type.columns = ['SK_ID_CURR'] + [i[0]+"_"+i[1]+"_"+pref+"_" f'l{lxm_counts}m' for i in amt_type.columns[1:]]
    ts_result = pd.merge(amt_type, count_type, how='outer', on=['SK_ID_CURR'])

    all_count_col = [i for i in ts_result.columns if 'no_' in i]
    all_amt_col = [i for i in ts_result.columns if 'amt_' in i]

    ts_result['no_tot_'+pref+'_'+ f'l{lxm_counts}m'] = ts_result[all_count_col].sum(axis=1)
    ts_result['amt_tot_'+pref+'_'+ f'l{lxm_counts}m'] = ts_result[all_amt_col].sum(axis=1)
    return ts_result

def gen_ts(df_ts, lxm_list = [3,6,9]):
    """
    """
    result_cal_ts = []
    ft_status1 = df_ts['is_active']==1
    ft_status2 = df_ts['is_active']==0
    ft_status3 = df_ts['is_active'].isin([0,1])

    ft_status_lst = [ft_status1, ft_status2, ft_status3]
    status_prefix = ['active', 'closed', 'all']
    for lxm_counts in tqdm(lxm_list):
        for ft_status, pref in zip(ft_status_lst, status_prefix):
            ft_lxm = (df_ts['LxM']*-1) <= lxm_counts
            ts_ft = df_ts[ft_status & ft_lxm]
            result_cal_ts.append(cal_ts(ts_ft, pref, lxm_counts))  
    conso = reduce(lambda  left,right: pd.merge(left, right, on=group_col, how='outer'), result_cal_ts)
    return conso

In [ ]:
df_ts_amt_n_count = gen_ts(df_ts, lxm_list=[3,6,9,12,18,26,36,48])

In [ ]:
filter1ts = distinct_count(df_ts_amt_n_count)
drop_most_count = list(set(filter1ts[filter1ts['most_count']>=0.8]['col'])-set(id_col))
drop_most_null = list(set(filter1ts[filter1ts['null_count']>=0.6]['col'])-set(id_col))
to_drop = drop_most_count+drop_most_null
to_drop = list(set(to_drop))

In [ ]:
len(to_drop)

In [ ]:
to_keep = list(set(df_ts_amt_n_count.columns) - set(to_drop))

In [ ]:
len(set(to_keep))

In [ ]:
df_ts_amt_n_count_fn = df_ts_amt_n_count[list(set(['SK_ID_CURR']+to_keep))]

In [ ]:
df_ts_amt_n_count_fn.to_parquet(r'data/df_ts_amt_n_count_fn.parquet')

#### Checkpoint2

In [25]:
df_ln_grp = pd.read_parquet(r'/kaggle/input/homecredit-feature-group/df_ln_grp.parquet')
df_pcb_cr_fn = pd.read_parquet(r'/kaggle/input/homecredit-feature-group/df_pcb_cr_fn.parquet')
df_ts_amt_n_count_fn = pd.read_parquet(r'/kaggle/input/homecredit-feature-group/df_ts_amt_n_count_fn.parquet')

In [20]:
df_ln_grp.shape

(89696, 4)

In [21]:
df_pcb_cr_fn.shape

(260395, 21)

In [22]:
df_ts_amt_n_count_fn.shape

(91767, 176)

## Installments

In [28]:
installments_payments[installments_payments['SK_ID_CURR']==161674].sort_values(by=['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER']).head(5)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,LxM
577850,1054186,161674,1.0,1,-1330.0,-1338.0,6948.36,6948.36,-44.0
47438,1054186,161674,1.0,2,-1300.0,-1307.0,6948.36,6948.36,-43.0
425089,1054186,161674,1.0,3,-1270.0,-1275.0,6948.36,6948.36,-42.0
3612113,1054186,161674,1.0,4,-1240.0,-1247.0,6948.36,6948.36,-41.0
1878907,1054186,161674,1.0,5,-1210.0,-1217.0,6948.36,6948.36,-40.0


In [34]:
installments_payments[installments_payments['DAYS_INSTALMENT']<installments_payments['DAYS_ENTRY_PAYMENT']]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,LxM,dpd
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,-45.0,17.0
19,2329072,183463,1.0,11,-1699.0,-1693.0,7283.295,7283.295,-56.0,6.0
30,1840786,175114,1.0,11,-476.0,-474.0,21179.700,21161.700,-16.0,2.0
45,1644499,111587,1.0,8,-2116.0,-2101.0,3078.810,3078.810,-69.0,15.0
47,1895324,131465,1.0,6,-1386.0,-1379.0,3229.065,3212.865,-45.0,7.0
...,...,...,...,...,...,...,...,...,...,...
13605308,1493367,415806,1.0,18,-587.0,-586.0,28873.170,28873.170,-19.0,1.0
13605319,2003162,450999,1.0,16,-2135.0,-2113.0,5414.265,5384.880,-70.0,22.0
13605328,2608958,408941,1.0,18,-41.0,-40.0,22788.135,22788.135,-1.0,1.0
13605340,1635645,417079,1.0,1,-2221.0,-2215.0,9914.850,9010.485,-73.0,6.0


In [33]:
installments_payments['dpd'] = installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']

In [43]:
def flag_pmt_st(dpd, target):
    return 1 if dpd >= target else 0

In [44]:
installments_payments['is_late_5d'] =  installments_payments['dpd'].apply(lambda row: flag_pmt_st(row, 5))
installments_payments['is_late_10d'] =  installments_payments['dpd'].apply(lambda row: flag_pmt_st(row, 10))
installments_payments['is_late_20d'] =  installments_payments['dpd'].apply(lambda row: flag_pmt_st(row, 20))


installments_payments['is_early_5d'] =  installments_payments['dpd'].apply(lambda row: flag_pmt_st(row*-1, 5))
installments_payments['is_early_10d'] =  installments_payments['dpd'].apply(lambda row: flag_pmt_st(row*-1, 10))
installments_payments['is_early_20d'] =  installments_payments['dpd'].apply(lambda row: flag_pmt_st(row*-1, 20))

In [45]:
installments_payments.head(5)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,LxM,dpd,is_late_5d,is_late_10d,is_late_20d,is_early_5d,is_early_10d,is_early_20d
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,-39.0,-7.0,0,0,0,1,0,0
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,-71.0,0.0,0,0,0,0,0,0
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,-2.0,0.0,0,0,0,0,0,0
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,-79.0,-8.0,0,0,0,1,0,0
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,-45.0,17.0,1,1,0,0,0,0


In [24]:
installments_payments['LxM'] = round(installments_payments['DAYS_INSTALMENT']/30.5,0)

13605401

In [58]:
group_col = ['SK_ID_CURR','LxM']
agg_fn = ['sum']

ins_sum = {}
val_list = ['AMT_PAYMENT', 'AMT_INSTALMENT', 
            'is_late_5d', 'is_late_10d', 'is_late_20d',
           'is_early_5d', 'is_early_10d', 'is_early_20d']

for val in tqdm(val_list):
    ins_sum[val+'agg'] = agg_cal(installments_payments, group_col=group_col, val=val, agg_fn=agg_fn)

100%|██████████| 8/8 [01:03<00:00,  7.97s/it]


In [63]:
group_col = ['SK_ID_CURR','LxM']
agg_fn = ['count']
val = 'AMT_INSTALMENT'

ins_sum['count_no_contract'] = agg_cal(installments_payments, group_col=group_col, val=val, agg_fn=agg_fn)

In [70]:
to_merge = list(ins_sum.values())
ins_pmt_ts = reduce(lambda  left,right: pd.merge(left, right, on=['SK_ID_CURR','LxM'], how='outer'), to_merge)

In [73]:
ins_pmt_ts.columns = ['SK_ID_CURR', 'LxM', 'ins_pmt', 'ins_amt',
 'late_5d', 'late_10d', 'late_20d',
 'early_5d', 'early_10d', 'early_20d', 'no_ins_contract']

In [86]:
test = ['SK_ID_CURR', 'LxM', 'ins_pmt', 'ins_amt',
 'late_5d', 'late_10d', 'late_20d',
 'early_5d', 'early_10d', 'early_20d', 'no_ins_contract']

In [89]:
{i:a for i,a in zip(test,test)}

{'SK_ID_CURR': 'SK_ID_CURR',
 'LxM': 'LxM',
 'ins_pmt': 'ins_pmt',
 'ins_amt': 'ins_amt',
 'late_5d': 'late_5d',
 'late_10d': 'late_10d',
 'late_20d': 'late_20d',
 'early_5d': 'early_5d',
 'early_10d': 'early_10d',
 'early_20d': 'early_20d',
 'no_ins_contract': 'no_ins_contract'}

In [77]:
ins_pmt_ts.head()

,SK_ID_CURR,LxM,ins_pmt,ins_amt,late_5d,late_10d,late_20d,early_5d,early_10d,early_20d,no_ins_contract
0,100001,-96.0,3982.050,3982.050,0,0,0,0,0,0,1
1,100001,-95.0,3982.050,3982.050,1,1,0,0,0,0,1
2,100001,-94.0,3980.925,3980.925,0,0,0,0,0,0,1
3,100001,-56.0,3951.000,3951.000,0,0,0,1,0,0,1
4,100001,-55.0,3951.000,3951.000,0,0,0,1,1,1,1


In [82]:
list(set(ins_pmt_ts.columns) - set(group_col))[0]

'late_10d'

In [91]:
group_col = ['SK_ID_CURR']
val_col = ['ins_pmt', 'ins_amt',
 'late_5d', 'late_10d', 'late_20d',
 'early_5d', 'early_10d', 'early_20d', 'no_ins_contract']

agg_fn = ['sum', 'mean', 'max']
LxM=[3, 6, 9, 12, 18, 24, 30, 36, 40, 48, 54]
ins_pmt_feature = generate_feature_lxm(ins_pmt_ts, group_col=group_col,
                    val_col=val_col, agg_fn=agg_fn, LxM=LxM)

100%|██████████| 9/9 [01:01<00:00,  6.79s/it]


In [92]:
ins_pmt_feature.head()

,SK_ID_CURR,ins_pmt_sum_l3m,ins_pmt_mean_l3m,ins_pmt_max_l3m,ins_pmt_sum_l6m,ins_pmt_mean_l6m,ins_pmt_max_l6m,ins_pmt_sum_l9m,ins_pmt_mean_l9m,ins_pmt_max_l9m,ins_pmt_sum_l12m,ins_pmt_mean_l12m,ins_pmt_max_l12m,ins_pmt_sum_l18m,ins_pmt_mean_l18m,ins_pmt_max_l18m,ins_pmt_sum_l24m,ins_pmt_mean_l24m,ins_pmt_max_l24m,ins_pmt_sum_l30m,ins_pmt_mean_l30m,ins_pmt_max_l30m,ins_pmt_sum_l36m,ins_pmt_mean_l36m,ins_pmt_max_l36m,ins_pmt_sum_l40m,ins_pmt_mean_l40m,ins_pmt_max_l40m,ins_pmt_sum_l48m,ins_pmt_mean_l48m,ins_pmt_max_l48m,ins_pmt_sum_l54m,ins_pmt_mean_l54m,ins_pmt_max_l54m,ins_amt_sum_l3m,ins_amt_mean_l3m,ins_amt_max_l3m,ins_amt_sum_l6m,ins_amt_mean_l6m,ins_amt_max_l6m,...,early_20d_max_l40m,early_20d_sum_l48m,early_20d_mean_l48m,early_20d_max_l48m,early_20d_sum_l54m,early_20d_mean_l54m,early_20d_max_l54m,no_ins_contract_sum_l3m,no_ins_contract_mean_l3m,no_ins_contract_max_l3m,no_ins_contract_sum_l6m,no_ins_contract_mean_l6m,no_ins_contract_max_l6m,no_ins_contract_sum_l9m,no_ins_contract_mean_l9m,no_ins_contract_max_l9m,no_ins_contract_sum_l12m,no_ins_contract_mean_l12m,no_ins_contract_max_l12m,no_ins_contract_sum_l18m,no_ins_contract_mean_l18m,no_ins_contract_max_l18m,no_ins_contract_sum_l24m,no_ins_contract_mean_l24m,no_ins_contract_max_l24m,no_ins_contract_sum_l30m,no_ins_contract_mean_l30m,no_ins_contract_max_l30m,no_ins_contract_sum_l36m,no_ins_contract_mean_l36m,no_ins_contract_max_l36m,no_ins_contract_sum_l40m,no_ins_contract_mean_l40m,no_ins_contract_max_l40m,no_ins_contract_sum_l48m,no_ins_contract_mean_l48m,no_ins_contract_max_l48m,no_ins_contract_sum_l54m,no_ins_contract_mean_l54m,no_ins_contract_max_l54m
0,100002,71597.295,23865.765,53093.745,99352.620,16558.770000,53093.745,127107.945,14123.105000,53093.745,154863.270,12905.272500,53093.745,210373.920,11687.440000,53093.745,219625.695,11559.247105,53093.745,219625.695,11559.247105,53093.745,219625.695,11559.247105,53093.745,219625.695,11559.247105,53093.745,219625.695,11559.247105,53093.745,219625.695,11559.247105,53093.745,71597.295,23865.765,53093.745,99352.620,16558.770000,53093.745,...,1.0,9.0,0.473684,1.0,9,0.473684,1,3.0,1.0,1.0,6.0,1.000000,1.0,9.0,1.0,1.0,12.0,1.000000,1.0,18.0,1.000000,1.0,19.0,1.000000,1.0,19.0,1.000000,1.0,19.0,1.000000,1.0,19.0,1.000000,1.0,19.0,1.000000,1.0,19,1.000000,1
1,100006,116110.080,29027.520,29027.520,894979.530,127854.218571,720814.410,982062.090,98206.209000,720814.410,982062.090,98206.209000,720814.410,1007153.415,67143.561000,720814.410,1007153.415,67143.561000,720814.410,1007153.415,67143.561000,720814.410,1007153.415,67143.561000,720814.410,1007153.415,67143.561000,720814.410,1007153.415,67143.561000,720814.410,1007153.415,67143.561000,720814.410,116110.080,29027.520,29027.520,894979.530,127854.218571,720814.410,...,1.0,5.0,0.333333,1.0,5,0.333333,1,4.0,1.0,1.0,8.0,1.142857,2.0,11.0,1.1,2.0,11.0,1.100000,2.0,16.0,1.066667,2.0,16.0,1.066667,2.0,16.0,1.066667,2.0,16.0,1.066667,2.0,16.0,1.066667,2.0,16.0,1.066667,2.0,16,1.066667,2
2,100007,64150.560,16037.640,16037.640,112263.480,16037.640000,16037.640,160376.400,16037.640000,16037.640,208497.780,16038.290769,16046.100,324064.980,17056.051579,33019.200,445608.270,17824.330800,38995.290,631209.780,20361.605806,39188.385,744603.705,20683.436250,39188.385,744603.705,20683.436250,39188.385,755383.815,19368.815769,39188.385,776992.995,17266.511000,39188.385,64150.560,16037.640,16037.640,112263.480,16037.640000,16037.640,...,1.0,4.0,0.102564,1.0,4,0.088889,1,4.0,1.0,1.0,7.0,1.000000,1.0,10.0,1.0,1.0,13.0,1.000000,1.0,20.0,1.052632,2.0,27.0,1.080000,2.0,36.0,1.161290,2.0,42.0,1.166667,2.0,42.0,1.166667,2.0,47.0,1.205128,2.0,53,1.177778,2
3,100008,35761.950,17880.975,17885.835,89419.455,17883.891000,17885.835,143076.960,17884.620000,17885.835,178848.630,17884.863000,17885.835,737614.800,46100.925000,432218.295,788233.950,43790.775000,432218.295,788233.950,43790.775000,432218.295,788233.950,43790.775000,432218.295,788233.950,43790.775000,432218.295,788233.950,43790.775000,432218.295,788

In [97]:
# os.chdir(r'/kaggle/working/')
# ins_pmt_feature.to_parquet('ins_pmt_feature.parquet')

In [26]:
ins_pmt_feature = pd.read_parquet('ins_pmt_feature.parquet')

## POS_CASH balance

In [28]:
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [34]:
pos_cash_balance['LxM'] = pos_cash_balance['MONTHS_BALANCE']

In [35]:
pos_cash_balance[pos_cash_balance['status_grp']==1].head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,status_grp,LxM
0,1803195,182943,-31,48.0,45.0,Active,0,0,1,-31
1,1715348,367990,-33,36.0,35.0,Active,0,0,1,-33
2,1784872,397406,-32,12.0,9.0,Active,0,0,1,-32
3,1903291,269225,-35,48.0,42.0,Active,0,0,1,-35
4,2341044,334279,-35,36.0,35.0,Active,0,0,1,-35


In [31]:
pos_cash_balance['status_grp'] = np.where(pos_cash_balance['NAME_CONTRACT_STATUS']=='Active', 1,
                                         np.where(pos_cash_balance['NAME_CONTRACT_STATUS']=='Completed',0,2)
                                         )

In [51]:
group_col = ['SK_ID_CURR']
val_col = ['CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE',
          'SK_DPD', 'SK_DPD_DEF']
agg_fn = ['sum', 'mean', 'max']
LxM=[3, 6, 9, 12, 18, 24, 30, 36, 40, 48, 54]

pos_cash_result = {}
pos_bal_df_lst = [pos_cash_balance[pos_cash_balance['status_grp']==1],
                  pos_cash_balance[pos_cash_balance['status_grp']==0],
                  pos_cash_balance
                 ]
df_name = ['Active', 'Completed', 'All']
for pos_bal_df, key in zip(pos_bal_df_lst, df_name):
    pos_cash_result[key] = generate_feature_lxm(pos_bal_df, group_col=group_col,
                    val_col=val_col, agg_fn=agg_fn, LxM=LxM)

100%|██████████| 4/4 [00:44<00:00, 11.21s/it]


In [53]:
act_rename = {f:'act_'+f for f in set(pos_cash_result['Active'].columns)- set(['SK_ID_CURR'])}
comp_rename = {f:'comp_'+f for f in set(pos_cash_result['Completed'].columns)- set(['SK_ID_CURR'])}
pos_cash_result['Active'].rename(columns=act_rename, inplace=True)
pos_cash_result['Completed'].rename(columns=comp_rename, inplace=True)

In [55]:
to_merge = list(pos_cash_result.values())
pos_cas_ts = reduce(lambda  left,right: pd.merge(left, right, on=['SK_ID_CURR'], how='outer'), to_merge)

In [57]:
pos_cas_ts.shape

(317408, 397)

## Credit card balance

In [58]:
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


## Previous application

# 3. Feature selection

Univarite <br>
Multivariate

In [ ]:

"""
> Filter selection.
> Marginal IV.
> Stepwise regression for model selecton.
> Resursive feature selection.
> 
> For ML?
"""

# 4. Model training

In [ ]:
"""
Tunning
"""

# 5. Evaluation 
1. residual analysis <br>
2. linear assumption <br>
3. assessing fit <br>
4. interaction among covariates <br>

# 6 Calibration

# 7. Model interpretation